# Profiling ANDES in Notebook

## Profiling with Python CProfiler

Before getting started, this example requires the config flag `PFlow.init_tds` to be `0`, which is the default value.

In [1]:
import andes
from andes.utils.paths import get_case

case_path = get_case('kundur/kundur_full.xlsx')

Passing `profile=True, no_output = True` to `run` will enable the profiler and have the results printed.

In [2]:
ss = andes.run(case_path, profile=True, routine='tds', no_output=True)

Working directory: "/Users/hcui7/repos/andes/examples"
Loaded config from file "/Users/hcui7/.andes/andes.rc"
Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"
Input file parsed in 0.0868 seconds.

-> Power flow calculation
Sparse Solver: KLU
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60858   
2: |F(x)| = 0.17009   
3: |F(x)| = 0.00203822
4: |F(x)| = 3.76399e-07
Converged in 5 iterations in 0.0090 seconds.

-> Time Domain Simulation Summary:
Sparse Solver: KLU
Simulation time: 0-20.0 sec.
Fixed step size: h=33.33 msec., shrink if not converged
Initialization was successful in 0.0202 seconds.


<Toggler 1>: Line.Line_8 status changed to 0 at t=2.0 sec.            
100%|████████████████████████████████| 100/100 [00:01<00:00, 63.79%/s]

Simulation completed in 1.5681 seconds.

-> Single process finished in 1.8119 seconds.



         1129976 function calls (1124199 primitive calls) in 1.789 seconds

   Ordered by: cumulative time
   List reduced from 2169 to 40 due to restriction <40>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.007    0.007    1.597    1.597 /Users/hcui7/repos/andes/andes/routines/tds.py:160(run)
      602    0.104    0.000    1.533    0.003 /Users/hcui7/repos/andes/andes/routines/tds.py:251(_itm_step)
     2318    0.010    0.000    1.141    0.000 /Users/hcui7/repos/andes/andes/routines/tds.py:590(_fg_update)
    11649    0.092    0.000    1.085    0.000 /Users/hcui7/repos/andes/andes/system.py:1014(call_models)
     2323    0.002    0.000    0.596    0.000 /Users/hcui7/repos/andes/andes/system.py:700(g_update)
    23210    0.183    0.000    0.556    0.000 /Users/hcui7/repos/andes/andes/core/model.py:1194(g_update)
     2323    0.002    0.000    0.187    0.000 /Users/hcui7/repos/andes/andes/system.py:686(f_update)
        1    0.000    0.000    

## Profiling with `line_profiler`.

`line_profiler` provides line-based profiling results for functions. 

Install with `pip install line_profiler` and restart the notebook.

In [3]:
import andes
from andes.utils.paths import get_case

case_path = get_case('kundur/kundur_full.xlsx')

### Profile power flow 

Pass the function name to profile to the magic `%lprun`, followed by a call to the function itself or an upper-level function.

Results will be shown in a popup window.

In [4]:
%lprun -f andes.routines.pflow.PFlow.run andes.run(case_path, no_output=True)

Working directory: "/Users/hcui7/repos/andes/examples"
Loaded config from file "/Users/hcui7/.andes/andes.rc"
Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"
Input file parsed in 0.1257 seconds.

-> Power flow calculation
Sparse Solver: KLU
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60858   
2: |F(x)| = 0.17009   
3: |F(x)| = 0.00203822
4: |F(x)| = 3.76399e-07
Converged in 5 iterations in 0.0133 seconds.
-> Single process finished in 0.2882 seconds.


Alternatively, do

In [5]:
ss = andes.run(case_path, no_output=True)

Working directory: "/Users/hcui7/repos/andes/examples"
Loaded config from file "/Users/hcui7/.andes/andes.rc"
Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"
Input file parsed in 0.0625 seconds.

-> Power flow calculation
Sparse Solver: KLU
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60858   
2: |F(x)| = 0.17009   
3: |F(x)| = 0.00203822
4: |F(x)| = 3.76399e-07
Converged in 5 iterations in 0.0084 seconds.
-> Single process finished in 0.1422 seconds.


In [6]:
ss.reset()
%lprun -f ss.PFlow.run ss.PFlow.run()


-> Power flow calculation
Sparse Solver: KLU
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60858   
2: |F(x)| = 0.17009   
3: |F(x)| = 0.00203822
4: |F(x)| = 3.76399e-07
Converged in 5 iterations in 0.0107 seconds.


To dig into the Newton Raphson iteration steps, profile each step instead with:

In [7]:
ss.reset()
%lprun -f ss.PFlow.nr_step ss.PFlow.run()


-> Power flow calculation
Sparse Solver: KLU
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60858   
2: |F(x)| = 0.17009   
3: |F(x)| = 0.00203822
4: |F(x)| = 3.76399e-07
Converged in 5 iterations in 0.0107 seconds.


### Profile time-domain simulation

In [8]:
%lprun -f ss.TDS._itm_step ss.TDS.run()


-> Time Domain Simulation Summary:
Sparse Solver: KLU
Simulation time: 0-20.0 sec.
Fixed step size: h=33.33 msec., shrink if not converged
Initialization was successful in 0.0290 seconds.


<Toggler 1>: Line.Line_8 status changed to 0 at t=2.0 sec.            
100%|████████████████████████████████| 100/100 [00:01<00:00, 51.13%/s]

Simulation completed in 1.9561 seconds.


## Cleanup

In [9]:
!andes misc -C


    _           _         | Version 1.0.7.post68.dev0+gc2c8333f
   /_\  _ _  __| |___ ___ | Python 3.7.1 on Darwin, 07/28/2020 10:40:52 PM
  / _ \| ' \/ _` / -_|_-< | 
 /_/ \_\_||_\__,_\___/__/ | This program comes with ABSOLUTELY NO WARRANTY.

"/Users/hcui7/repos/andes/examples/kundur_full_out.txt" removed.
"/Users/hcui7/repos/andes/examples/kundur_out.txt" removed.
"/Users/hcui7/repos/andes/examples/kundur_full_out.npz" removed.
"/Users/hcui7/repos/andes/examples/kundur_out.npz" removed.
"/Users/hcui7/repos/andes/examples/kundur_full_out.csv" removed.
"/Users/hcui7/repos/andes/examples/kundur_full_out.lst" removed.
"/Users/hcui7/repos/andes/examples/kundur_out.lst" removed.
